In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_episodes'

Primer paso: inspecciona en tu navegador el link:

https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_episodes

In [3]:
r = requests.get(url)

In [4]:
print(r.text[:500])


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of Game of Thrones episodes - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ee9bf5ec-1bd2-423d-806


In [5]:
html_contents = r.text

In [6]:
html_soup = BeautifulSoup(html_contents, 'lxml') # html.parser

Algunas cosillas más ...

In [7]:
first_h1 = html_soup.find("h1")
print(first_h1)

<h1 class="firstHeading" id="firstHeading" lang="en">List of <i>Game of Thrones</i> episodes</h1>


In [8]:
print(first_h1.text)
print(first_h1.get_text()) # hace lo mismo

List of Game of Thrones episodes
List of Game of Thrones episodes


In [9]:
print(first_h1.attrs)
print(first_h1.attrs['id'])
print(first_h1['id']) # hace lo mismo
print(first_h1.get('id')) # hace lo mismo

{'id': 'firstHeading', 'class': ['firstHeading'], 'lang': 'en'}
firstHeading
firstHeading
firstHeading


Ahora saca la lista de los diferentes "seasons". En este caso, debería dar algo como:


['2.1 Season 1 (2011)',

'2.2 Season 2 (2012)',

'2.3 Season 3 (2013)',

'2.4 Season 4 (2014)',

'2.5 Season 5 (2015)',

'2.6 Season 6 (2016)',

'2.7 Season 7 (2017)',

'2.8 Season 8']

In [10]:
# primera forma de hacerlo:

[x.text
 for x in html_soup.find_all("li", class_="toclevel-2")]

['2.1 Season 1 (2011)',
 '2.2 Season 2 (2012)',
 '2.3 Season 3 (2013)',
 '2.4 Season 4 (2014)',
 '2.5 Season 5 (2015)',
 '2.6 Season 6 (2016)',
 '2.7 Season 7 (2017)',
 '2.8 Season 8 (2019)']

In [11]:
# Segunda forma de hacerlo:

aux1 = html_soup.find("a", {"href":"#Episodes"})
aux2 = aux1.find_next_sibling("ul")
[x.text for x in aux2.find_all("li")]

['2.1 Season 1 (2011)',
 '2.2 Season 2 (2012)',
 '2.3 Season 3 (2013)',
 '2.4 Season 4 (2014)',
 '2.5 Season 5 (2015)',
 '2.6 Season 6 (2016)',
 '2.7 Season 7 (2017)',
 '2.8 Season 8 (2019)']

Ahora selecciona la tabla correspondiente a "Season 1" y calcula cuántas filas (tags "tr") te salen.
Calcula también cuántas celdas (tags "td") te salen.

¿Te cuadra? ¿por qué? Ayuda: Inspecciona el contenido html página en tu navegador

In [12]:
table = html_soup.find("table", class_="wikiepisodetable")
len(table.find_all("tr")), len(table.find_all("td"))

(11, 60)

Ahora saca la lista de directores del primer "episode":

In [13]:
directores = []
filas = table.find_all("tr")[1:]
for fila in filas:
    directores.append(fila.find_all("td")[2].text)
list(set(directores))

['Daniel Minahan', 'Brian Kirk', 'Alan Taylor', 'Tim Van Patten']

In [14]:
# Otra forma (más eficiente porque no extrae todas las celdas
# de la fila):

directores = []
filas = table.find_all("tr")[1:]
for fila in filas:
    aux1 = fila.find("td") # primera celda td en fila
    aux2 = aux1.find_next_sibling("td") # la segunda
    aux3 = aux2.find_next_sibling("td") # la tercera
    directores.append(aux3.text)
list(set(directores))

['Daniel Minahan', 'Brian Kirk', 'Alan Taylor', 'Tim Van Patten']

Ahora saca la lista de directores del primer "episode" junto a sus enlaces en Wikipedia:

In [15]:
def get_domain_name(x):
    aux = x.split("//")
    aux[-1] = aux[-1].split("/")[0]
    return "//".join(aux)

def completar_link(link, url_pagina_actual):
    if link[:4] != 'http':
        if link[:2] == '//':
            link = 'http:' + link
        elif link[0] == '/':
            domain_name = get_domain_name(url_pagina_actual)
            link = domain_name + link
        else:
            link = url_pagina_actual + link
    return link

In [16]:
aux = "https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_episodes"
completar_link('/wiki/Tim_Van_Patten', aux)

'https://en.wikipedia.org/wiki/Tim_Van_Patten'

In [17]:
directores = {}
filas = table.find_all("tr")[1:]

for fila in filas:
    aux1 = fila.find("td") # primera celda td en fila
    aux2 = aux1.find_next_sibling("td") # la segunda
    aux3 = aux2.find_next_sibling("td") # la tercera
    director = aux3.text
    
    if aux3.find("a") is None: # el link (href) está dentro de <a ...> y si
        # no tiene <a ...> no tiene sentido extraer el link
        continue # pasamos a la siguiente itaración del bucle
    
    link = aux3.find("a").get("href")
    link = completar_link(link, url)
    
    if director not in directores.keys():
        directores[director] = link

directores

{'Tim Van Patten': 'https://en.wikipedia.org/wiki/Tim_Van_Patten',
 'Brian Kirk': 'https://en.wikipedia.org/wiki/Brian_Kirk',
 'Daniel Minahan': 'https://en.wikipedia.org/wiki/Daniel_Minahan',
 'Alan Taylor': 'https://en.wikipedia.org/wiki/Alan_Taylor_(director)'}

In [18]:
directores.keys()

dict_keys(['Tim Van Patten', 'Brian Kirk', 'Daniel Minahan', 'Alan Taylor'])

**Nota: hay que completar los links!**

Se continúa la solución en notebook 02-juego_de_tronos_SOLUCION_CON_LIBRERIA.ipynb donde además se usa **mi_libreria.py**

Ahora saca la lista de todos los directores que ha habido en cualquier "episode" de Game of Thrones:

Ahora saca por pantalla el título y link de las 5 primeras citas con clase "citation" de la sección "Referencias":